In [1]:
import torch
from torch import nn
import data_utils
from training.MADGAN_train import train_kdd99, train_financial
from models.MADGAN import Generator, Discriminator, AnomalyDetector
from utils import evaluation

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# Hyperparameters

In [3]:
# choose dataset (kdd99, aapl, gm, axp)
dataset = "kdd99"

In [4]:
num_features = 6
seq_len = 30
seq_stride = 10
gen_seq_len = 1

random_seed = 0
num_epochs = 100
batch_size = 256
lr = 1e-5
wd = 5e-7

latent_dim = 250
hidden_dim = 500
anomaly_threshold = 0.5

# Load data

In [5]:
if dataset == "kdd99":
    train_dl, test_dl = data_utils.kdd99(seq_len, seq_stride, num_features, gen_seq_len, batch_size)
else:
    file_path = 'data/financial_data/Stocks/'+dataset+'.us.txt'
    tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_len, seq_stride, gen_seq_len, batch_size, normalise=True)

# Model

In [6]:
generator = Generator(input_dim=latent_dim,hidden_size=hidden_dim,output_dim=num_features).to(device=DEVICE)

In [7]:
discriminator = Discriminator(input_dim=num_features,hidden_size=hidden_dim).to(device=DEVICE)

# Loss and Optimizer

In [8]:
def loss_function(inputs, targets):
    return nn.BCELoss()(inputs, targets)

In [9]:
discriminator_optim = torch.optim.Adam(discriminator.parameters(), lr=lr, weight_decay=wd)
generator_optim = torch.optim.Adam(generator.parameters(), lr=lr, weight_decay=wd)

# Train

In [10]:
if dataset == "kdd99":
    train_kdd99(seq_len, latent_dim, train_dl, test_dl, discriminator, generator, discriminator_optim, generator_optim, anomaly_threshold, loss_function, random_seed, num_epochs, DEVICE)
else:
    train_financial(seq_len, latent_dim, tscv_dl_list, discriminator, generator, discriminator_optim, generator_optim, anomaly_threshold, loss_function, random_seed, num_epochs, DEVICE)

Epoch 0: G_loss: 0.834706754305146, D_loss_real: 0.7151220870288936, D_loss_fake: 0.6353746102614837
Epoch 1: G_loss: 0.6407460927963257, D_loss_real: 0.6141079519282687, D_loss_fake: 0.8036763776432384
Epoch 2: G_loss: 0.8770153690468181, D_loss_real: 0.8175321188839999, D_loss_fake: 0.6102429064837369
Epoch 3: G_loss: 0.6864724587310445, D_loss_real: 0.75492042533376, D_loss_fake: 0.76549877632748
Epoch 4: G_loss: 0.7223025259646503, D_loss_real: 0.7027616679668427, D_loss_fake: 0.7008117632432418
Epoch 5: G_loss: 0.7623504264788195, D_loss_real: 0.72283129515973, D_loss_fake: 0.6588065790859136
Epoch 6: G_loss: 0.7509782869707454, D_loss_real: 0.7098982229828834, D_loss_fake: 0.6921655337918888
Epoch 7: G_loss: 0.7224797847596082, D_loss_real: 0.7118286946957761, D_loss_fake: 0.7187288663604042
Epoch 8: G_loss: 0.8063274031335658, D_loss_real: 0.7319640003822067, D_loss_fake: 0.6517469118941914
Epoch 9: G_loss: 0.7026590691371397, D_loss_real: 0.6978575407104058, D_loss_fake: 0.7241

Epoch 80: G_loss: 0.6378402905030685, D_loss_real: 0.6419393097812479, D_loss_fake: 0.7559606419368223
Epoch 81: G_loss: 0.7720506765625693, D_loss_real: 0.7429974982345646, D_loss_fake: 0.6455792397260666
Epoch 82: G_loss: 0.7789904128421437, D_loss_real: 0.7456658850339326, D_loss_fake: 0.656325071237304
Epoch 83: G_loss: 0.6333510033108971, D_loss_real: 0.588413405418396, D_loss_fake: 0.765833830833435
Epoch 84: G_loss: 0.8379460890184749, D_loss_real: 0.7994682520627976, D_loss_fake: 0.6037083047357472
Epoch 85: G_loss: 0.683508887887001, D_loss_real: 0.6900831845673647, D_loss_fake: 0.7061869065869938
Epoch 86: G_loss: 0.669508574496616, D_loss_real: 0.6561953169378367, D_loss_fake: 0.723998271335255
Epoch 87: G_loss: 0.7133125110106034, D_loss_real: 0.7090925327756188, D_loss_fake: 0.6809619158506394
Epoch 88: G_loss: 0.7305170164866881, D_loss_real: 0.7115715614774011, D_loss_fake: 0.6723040488633243
Epoch 89: G_loss: 0.6951997640458021, D_loss_real: 0.6663291107524525, D_loss_f

In [23]:
from utils.evaluation import accuracy, precision, recall, metric_calc
from training.MADGAN_train import emmv
def evaluate(model, test_dl, label, DEVICE):
    total_em = total_mv = total_acc = total_pre = total_rec = 0
    total_time = 0
    for X, Y in test_dl:
        X = X.to(DEVICE)
        prediction = model.predict(X)[:, :Y.size(1), :]
        true_positives, true_negatives, false_positives, false_negatives = metric_calc(prediction.view(-1, 1), Y.view(-1, 1), label)
        total_acc += accuracy(true_positives, true_negatives, Y)
        if (true_positives+false_positives) > 0:
            total_pre += precision(true_positives, false_positives)
        if (true_positives+false_negatives) > 0:
            total_rec += recall(true_positives, false_negatives)
        em, mv = emmv(model, X.to(DEVICE), DEVICE=DEVICE)
        total_mv += mv
        total_em += em
    print("Detection time: {0}".format(total_time))
    print("Acc: {0}, Pre: {1}, Rec: {2}".format(total_acc/len(test_dl), total_pre/len(test_dl), total_rec/len(test_dl)))
    print("EM: {0}, MV: {1}".format(total_em/len(test_dl), total_mv/len(test_dl)))

In [24]:
#from training.MADGAN_train import evaluate
anomaly_threshold = 0.4
ad = AnomalyDetector(discriminator=discriminator, generator=generator, latent_space_dim=latent_dim, anomaly_threshold=anomaly_threshold, DEVICE=DEVICE)
evaluate(ad, test_dl, label=1, DEVICE=DEVICE)

Detection time: 0
Acc: 0.8021946513385145, Pre: 0.8029482309430644, Rec: 0.9472784247829995
EM: 0.000171587113127936, MV: 13085.227405745043
